In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from osgeo import gdal
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/ubuntu/anaconda3/envs/hydra/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning:

pylab import has clobbered these variables: ['Annotation', 'Figure']
`%matplotlib` prevents importing * from pylab and numpy



In [3]:
atms=gdal.Open('../data/test/SATMS_npp_d20180308_t1255440_e1303436_b32960_c20180308190342349213_nobc_ops.h5', gdal.GA_ReadOnly)
subdata2 = atms.GetSubDatasets()
subdata2

AttributeError: 'NoneType' object has no attribute 'GetSubDatasets'

In [ ]:
BT = gdal.Open('HDF5:"../data/test/SATMS_npp_d20180308_t1255440_e1303436_b32960_c20180308190342349213_nobc_ops.h5"://All_Data/ATMS-SDR_All/BrightnessTemperature')
GainCal = gdal.Open('HDF5:"../data/test/SATMS_npp_d20180308_t1255440_e1303436_b32960_c20180308190342349213_nobc_ops.h5"://All_Data/ATMS-SDR_All/GainCalibration')
NEdTc = gdal.Open('HDF5:"../data/test/SATMS_npp_d20180308_t1255440_e1303436_b32960_c20180308190342349213_nobc_ops.h5"://All_Data/ATMS-SDR_All/NEdTCold')
NEdTw = gdal.Open('HDF5:"../data/test/SATMS_npp_d20180308_t1255440_e1303436_b32960_c20180308190342349213_nobc_ops.h5"://All_Data/ATMS-SDR_All/NEdTWarm')
QF20 = gdal.Open('HDF5:"../data/test/SATMS_npp_d20180308_t1255440_e1303436_b32960_c20180308190342349213_nobc_ops.h5"://All_Data/ATMS-SDR_All/QF20_ATMSSDR')
QF21 = gdal.Open('HDF5:"../data/test/SATMS_npp_d20180308_t1255440_e1303436_b32960_c20180308190342349213_nobc_ops.h5"://All_Data/ATMS-SDR_All/QF21_ATMSSDR')
QF22 = gdal.Open('HDF5:"../data/test/SATMS_npp_d20180308_t1255440_e1303436_b32960_c20180308190342349213_nobc_ops.h5"://All_Data/ATMS-SDR_All/QF22_ATMSSDR')
lat = gdal.Open('HDF5:"../data/test/GATMO_npp_d20180308_t1255440_e1303436_b32960_c20180308190342336760_nobc_ops.h5"://All_Data/ATMS-SDR-GEO_All/Latitude')
lon = gdal.Open('HDF5:"../data/test/GATMO_npp_d20180308_t1255440_e1303436_b32960_c20180308190342336760_nobc_ops.h5"://All_Data/ATMS-SDR-GEO_All/Longitude')
view = gdal.Open('HDF5:"../data/test/GATMO_npp_d20180308_t1255440_e1303436_b32960_c20180308190342336760_nobc_ops.h5"://All_Data/ATMS-SDR-GEO_All/SatelliteZenithAngle')

In [ ]:
#BT and other data in array format
BTr = BT.ReadAsArray()
GainCalr = GainCal.ReadAsArray()
NEdTcr = NEdTc.ReadAsArray()
NEdTwr = NEdTw.ReadAsArray()
QF20r = QF20.ReadAsArray()
QF21r = QF21.ReadAsArray()
QF22r = QF22.ReadAsArray()
lats = lat.ReadAsArray()
lons = lon.ReadAsArray()
views = view.ReadAsArray()

In [ ]:
ATMS_sub = gdal.Open(atms.GetSubDatasets()[1][0], gdal.GA_ReadOnly)
ATMS_array = ATMS_sub.ReadAsArray()
print(QF20r.shape)
GainCalr.shape

In [ ]:
imshow(np.flipud(views)>50)

In [ ]:
test = np.zeros_like(BTr).astype(np.uint16)
for i in range(BTr.shape[2]):
    for j in range(BTr.shape[0]):
        test[j,:,i] = (BTr[j,:,i].astype(int) * GainCalr[j,i]) *0.0001

In [ ]:
print(BTr.shape,lats.shape)

import cartopy.crs as ccrs
ax = pl.axes(projection=ccrs.PlateCarree())
plt.pcolormesh(lons, lats, test[:,:,0]>170,transform=ccrs.PlateCarree(),cmap='Blues_r')

ax.coastlines(color='gray')

In [ ]:
df = pd.read_hdf('../data/cal/atms_mlc_coeffs.hdf','table')
cols = ['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19','C20','C21','C22',]
df = pd.DataFrame(df,columns = cols)

coeffs = df.to_dict()

keys = list(coeffs.keys())
print(keys)

cal_mlc = np.zeros_like(test)

for c in range(len(keys)):
    chnl = keys[c]
    tbi = coeffs[chnl]['Tbi']
    preds = coeffs[chnl]['predictors']
    popt = coeffs [chnl]['popt']
    for i in range(cal_mlc.shape[0]):
        factor = np.zeros(cal_mlc.shape[1])
        for j in range(popt.size):
            factor = factor + (popt[j]*(test[i,:,preds[j]]-coeffs[keys[j]]['Tbij']))
        cal_mlc[i,:,c] = tbi + factor

In [ ]:
imshow(np.flipud(test[:,:,3]))
colorbar()
title('Raw ATMS - Channel 4')

In [ ]:
imshow(np.flipud(cal_mlc[:,:,3]))
colorbar()
title('Microwave Limb Corrected - Channel 4')

In [ ]:
transO = np.mean(test[:,:,3],axis=0)
transT = np.mean(cal_mlc[:,:,3],axis=0)

plot(transO)
plot(transT)

In [ ]:
rain = cal_mlc[:,:,15] - cal_mlc[:,:,0]
ax = pl.axes(projection=ccrs.PlateCarree())
plt.pcolormesh(lons, lats, rain>10,transform=ccrs.PlateCarree(),cmap='Blues_r')

ax.coastlines(color='gray')

In [ ]:
rain = cal_mlc[:,:,15] - cal_mlc[:,:,0]
plt.imshow(np.flipud(rain)<10)

In [ ]:
import pysptools.util as util
import pysptools.eea as eea
import pysptools.abundance_maps as amp

In [ ]:
btDiff = (cal_mlc[:,:,3].astype(np.float)-cal_mlc[:,:,2])
idx = np.where(views<50)
print(idx)
samps = btDiff[idx]
print(btDiff.shape)
samps = samps[:,np.newaxis,np.newaxis]

nfindr = eea.NFINDR()
U = nfindr.extract(samps, 2, maxit=50, normalize=True, ATGP_init=True)

In [ ]:
nfindr.display()

In [ ]:
nnls = amp.NNLS()
amaps = nnls.map(btDiff[:,:,np.newaxis], U, normalize=True)
print(U)

In [ ]:
ax = pl.axes(projection=ccrs.PlateCarree())
plt.pcolormesh(lons, lats, amaps[:,:,0],transform=ccrs.PlateCarree())


In [ ]:
fWater = np.ma.masked_where(views>50,amaps[:,:,0])
fWater = np.ma.masked_where(rain>10,fWater)

fig, ax = plt.subplots()
ax = pl.axes(projection=ccrs.PlateCarree())
cb = plt.pcolormesh(lons, lats, fWater,transform=ccrs.PlateCarree(),cmap='Blues')
ax.coastlines(color='gray')

cax = fig.add_axes([0.125, 0.05, 0.775, 0.02])
fig.colorbar(cb, cax=cax, orientation='horizontal')
